In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

### Importing the dataset

In [ ]:
# read train.EN.csv into a dataframe
df = pd.read_csv('https://raw.githubusercontent.com/iabufarha/iSarcasmEval/main/train/train.En.csv', index_col=False)

# read test.EN.csv into a dataframe
df_test = pd.read_csv('https://raw.githubusercontent.com/iabufarha/iSarcasmEval/main/test/task_A_En_test.csv', index_col=False)

In [ ]:
df1 = df[['tweet', 'sarcastic']]
df2 = df_test[['text', 'sarcastic']]

In [ ]:
df1.head(1)

,tweet,sarcastic
0,The only thing I got from college is a caffein...,1


In [ ]:
df2.head(1)

,text,sarcastic
0,"Size on the the Toulouse team, That pack is mo...",0


In [ ]:
batch_1 = df1

### Loading the Pre-trained BERT model

In [ ]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Model #1: DistilBERT

We first use DistilBERT to find the [CLS] tokens for all input sentences

### Tokenization

In [ ]:
tokenized = batch_1['tweet'].apply((lambda x: tokenizer.encode(str(x), add_special_tokens=True)))
tokenized_test = df2['text'].apply((lambda x: tokenizer.encode(str(x), add_special_tokens=True)))

### Padding

In [ ]:
# Pad to the same size for train data
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

# Pad to the same size for test data
max_len = 0
for i in tokenized_test.values:
    if len(i) > max_len:
        max_len = len(i)
      
padded_test = np.array([i + [0]*(max_len-len(i)) for i in tokenized_test.values])

Our dataset is now in the `padded` variable, we can view its dimensions below:

In [ ]:
print(np.array(padded).shape, np.array(padded_test).shape)

(3468, 116) (1400, 146)


### Masking

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_test_mask = np.where(padded_test != 0, 1, 0)
print(attention_mask.shape, attention_test_mask.shape)

(3468, 116) (1400, 146)


### Embedding

In [ ]:
# Use DistilBERT to embed all training data
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
# Use DistilBERT to embed all test data
input_ids_test = torch.tensor(padded_test)
attention_test_mask = torch.tensor(attention_test_mask)

with torch.no_grad():
    last_hidden_states_test = model(input_ids_test, attention_mask=attention_test_mask)

In [ ]:
# Save [CLS] tokens for classification
# last_hidden_states[0][sentences, token position [CLS] as the first one, 768 units]
train_features = last_hidden_states[0][:,0,:].numpy()
test_features = last_hidden_states_test[0][:,0,:].numpy()

In [ ]:
train_labels = batch_1['sarcastic']
test_labels = df2['sarcastic']

## Model #2: Logistic Regression

In [ ]:
# We now train the LogisticRegression model
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression()

## Evaluating Model

In [ ]:
lr_clf.score(test_features, test_labels)

0.8035714285714286

In [ ]:
pred_labels = lr_clf.predict(test_features)
f1_score(test_labels, pred_labels)

0.2028985507246377